# Using HBase from Python

(C) 2016 [Steve Phelps](http://sphelps.net)

### Overview

1. Column-oriented databases and Apache HBase
2. Binary data in Python 
3. Serialization using the `Struct` class.
4. The `happybase` API.

### Reading

- [no-SQL Data Modelling Techniques](https://highlyscalable.wordpress.com/2012/03/01/nosql-data-modeling-techniques/)
- [The HBase Data Model](https://hbase.apache.org/book.html#datamodel)
- [Tutorial on binary data in Python](https://pymotw.com/2/struct/)
- [The HappyBase User Guide](http://happybase.readthedocs.org/en/latest/user.html)

# Apache HBase

- HBase is a column-oriented database.
- It is inspired by Google's Bigtable \cite{Chang2006}.
- HBase is written in Java (Bigtable was written in C++).
- HBase can be used from Python through a [Thrift API](https://wiki.apache.org/hadoop/Hbase/ThriftApi).
- It can also be access through the [Apache Spark Python API](https://spark.apache.org/).

# Scalability

- The design philosophy is based on highly-distributed, [shared-nothing architecture](https://en.wikipedia.org/wiki/Shared_nothing_architecture).
- It provides automatic [sharding](https://en.wikipedia.org/wiki/Shard_\%28database_architecture\%29}{sharding}).
- HBase is part of the [Hadoop](https://hadoop.apache.org/) Ecosystem.

# Shared-nothing architecture

- When building scalable systems, we need to parallelize our computations.

- This requires multiple processors (and/or multiple cores).

- How do we distribute the data to each processor?


Scheme                | Description
------                | -----------
Shared memory (SM)    | Multiple processors share a common central memory.
Shared disk (SD)      | Multiple processors with private memory share a  common collection of disks.
Shared nothing (SN)   | Neither memory not peripheral storage is shared among processors.

\cite{Stonebraker1986}

# Sharding

- We can provide parallel access to a database by storing different subsets
of the data on different RDBMS servers.

- We could use, e.g. a hash function to determine which server a piece of data should reside on.

- Each of these subsets is called a shard.

- With a traditional relational database, sharding is very costly in terms of duplicated resources and the complexity of the configuration.

- HBase provides automatic sharding, with minimal duplication of state.

# Regions

- Tables are partitioned vertically into different *regions*.

- Different *region servers* are responsible for one or more regions.

- The HBase Master process performs coordination and load balancing across the region servers.

# The Hadoop Distributed File System (HDFS)

- In a production configuration HBase stores its underlying data on a file-system called [HDFS](https://hadoop.apache.org/docs/r1.0.4/hdfs_design.html#Introduction).

- HDFS uses a cluster of computers to simulate a single file-system.

- It provides:

    - Resilience against hardware 
    - Streaming Data Access
    - Support for large files (e.g. terabytes)
    
- It is based on the philosophy that moving computation is often cheaper than moving data.

# HBase Tables

- Tables are maps of maps.
- We first map from row key to a map of the data for that row.
- Then within each row, column names are mapped to values.
- Tables consist of:
    - *Rows*, which hold the data associated with the row-key.
    - *Column keys*, which are used to index each row.
    - *Row keys*, which are used to index each attribute within a row.
    - *Column families* which group related keys to specify access-control and options.

# Multi-dimensional sorted maps

- Row keys are sorted in [lexicographic order](http://mathworld.wolfram.com/LexicographicOrder.html}{lexicographic order).
- The number of columns per row is _unbounded_.
- This design implements a persistent, sparse, multi-dimensional sorted map.

# Sparse data

- When de-normalising a schema, we introduce many `NULL` values.
- A data-set which contains a significant fraction of `NULL` values is called `sparse` data.
- `NULL` values can be represented by the absence of a mapping.
- Existence is tested using a [Bloom filter](https://en.wikipedia.org/wiki/Bloom_filter).
- The bloom filter is cached in memory;
    - This prevents unnecessary disk accesses.

# HBase is typeless

- There are *no types*. 
- Both keys and their associated values are arbitrary-length arrays of bytes.
- There is no limit on the size of a value.

# Lexicographic ordering

- We can define an ordering over arbitrary binary data.

- For example, the following Python code determines whether $x < y$
according to lexicographic ordering:

In [5]:

def lexicographic_le(x, y):   
    for i in range(len(x)):
        if x[i] == y[i]:
            continue
        else:
            return x[i] < y[i]
    return False

lexicographic_le('steve', 'smith')

False

# Ordering arrays of bytes

In [6]:
import numpy as np

x = np.array([15, 9, 5, 16, 4])

y = np.array([15, 16, 1, 18, 1])

lexicographic_le(x, y)


True

# Joins in HBase

- There is no automatic enforcement of referentially integrity in a column-oriented database.
    - There are no foreign key constraints.

- If we want to retrieve the value associated with a particular key from another table, then
that will be a fast operation.

- In general, however, the set-theoretic operations available in SQL can be very expensive.

- Joins are not supported;
    we "pre-join" the data through de-normalisation.
    

# An example table

![hbase table](http://lwlink3.linkwithin.com/api/click?format=go&jsonp=vglnk_14684974959609&key=503c38809682907e0e07931326b1c03d&libId=iqm9hrid01012xfu000DAkt1oqmsc&loc=http%3A%2F%2Fgardenforkingpaths.blogspot.co.uk%2F2012%2F10%2Fhbase-column-family-store.html&v=1&out=http%3A%2F%2F1.bp.blogspot.com%2F-B8smblPeeqw%2FUHNjaBq-caI%2FAAAAAAAAAGU%2FHXMw8Hi2L90%2Fs1600%2Fhbase.png&ref=https%3A%2F%2Fduckduckgo.com&title=the%20Garden%20of%20Forking%20Paths%3A%20HBase%3A%20A%20Column-Family%20Store&txt=)

# Column qualifiers

- Row keys and column keys can consist of arbitrary binary data.

- Often we will use strings (which HBase will see an array of 8-bit ASCII values).

- Column-keys containing the character "`:`" specify membership of a column-family:

~~~
<column family name>:<column qualifer>
~~~

- The column family name must consist of printable text.

- For example, in the previous example, the `triangle` attribute would be written as:

`shape:triangle`

- The fully-qualified key is stored on disk.

- Therefore, the name of a column-family should generally be short, in order to maintain capacity and IO throughput.

# The colors and shapes example with Python dicts

In [8]:
data = \
    {
        'first': 
            {
                'color:red': 0xf00, 'color:blue': 0x00f, 
                    'shape:yellow': 0xff0, 'shape:square': 4 
            }, 
        'second': 
            { 
                'shape:square': 4, 'shape:triangle': 3 
            } 
    }

In [9]:
data['second']

{'shape:square': 4, 'shape:triangle': 3}

In [10]:
data['second']['shape:triangle']

3

# URLs

- BigTable was originally designed to house web data (it originated from Google).

- It is very common to use a URL as a row-key.

- We would typically reverse the URL before using it as a key.

- For example:

`keats.kcl.ac.uk` becomes `uk.ac.kcl.keats`.

# Time stamps

- HBase also allows multiple versions of the values associated with a given column for a given row.
- We can optionally use this to index by the time-stamp of the datum.
- For example, we might store several versions of a web page associated with a given URL.
- The time-stamp dimension is optionally.
- When writing data, if we do not specify the time-stamp, then the current system time is used.
- When modifying data, the old value is kept along with the old time-stamp.
- There are therefore three dimensions to the multi-dimensional map.
- For some applications, we can use the time-dimension to store our own data.

# Epoch time

- Time-stamps are stored as integers in [Unix epoch time](http://www.epochconverter.com/).
- Epoch time is number of seconds that have elapsed since 00:00:00 Coordinated Universal Time (UTC), Thursday, 1 January 1970
- To get the current epoch time on a Unix system:

~~~bash
date +%s
~~~

# Table operations

- There are four core CRUD operations which we can perform on tables:

    1. Get
    2. Put
    3. Delete
    4. Scan

- We execute them by either:
    - Executing a command in the shell.
    - Calling a Java method in the Java API.
    - Calling a Python function in the Thrift API.
    
- On a big project, it is typically more practical to use the API than the shell.

# Get

- Get returns attributes for a specified row-key and column(s).  

- It can be used in several ways:
    - Get everything for a row.
    - To get all columns from specific column families.
    - To get specific columns.
    - To get values within a specified time range.

- By default it will retrieve up to three versions of the data
    - We can specify a different number each time we get.

# Put

- Put can be used to insert new rows, or update existing rows.

- We always specify a row key.

- Where there is no existing value at a cell, new data is created.

- If there is existing data, it is retained under the old time stamp.

# Delete

- Delete a particular cell.

# Scan

- Scan is used to retrieve a *range* of rows in one operation.

- In its simplest form it will iterate over all rows in the table.

- We can also specify a minimum and maximum row-key.

- The ordering is given by the lexicographic ordering of the row-key itself.
    - Ascending and descending are supported.
    - Note that we cannot specify an arbitrary ordering of the data;
    - there is no equivalent of SQL's `ORDER BY` clause.
    

# Tall-Narrow Versus Flat-Wide tables

- In a column-oriented database we need to think carefully about the structure of the table in respect of the kinds of queries we want to support.

- We can choose a tall-narrow design as opposed to a flat-wide design.
    - Tall-narrow: very many rows, fewer columns
    - Flat-wide: fewer rows, very many columns
    
- A tall-narrow design is more readily shard-able.
    - Individual rows cannot be split across regions.
    
- A flat-wide design can be transformed into a tall-narrow design by storing additional data values in the row-key.

- These attributes can be retrieved by scanning with a *partial-key*.

# Partial Key Scans

- Because keys are sorted lexicographically, we can index several attributes simultaneously through concatenation.
- e.g. suppose we use row`keys as with the following format:

  `<userId>-<date>-<messageId>-<attachmentId>`
  
- We can then _scan_ the table by specifying a partial-key.

# Underlying data-structures.

- The ordering over keys is provided by Log-Structured Merge Trees \cite{Neil1996}.
- In contrast to [B+ Trees](https://en.wikipedia.org/wiki/B\%2B_tree) in traditional databases.
- LSMT trees ensure that rows of tables can efficiently be read from disk sequentially.
- The merging operation is scheduled in the background automatically.
- No need to constantly optimise tables to avoid fragmentation of pages.
- _Scanning_ a table over a large range is bound only by sequential IO latencies.

# CRUD operations from the shell

- We can perform create, read, update and delete (CRUD) operations from the HBase shell.

- To start the HBase shell type a command similar to the following from the Unix shell:

`hbase shell`

- To check the status of HBase use the command `status`.

## Types in HBase

- HBase does not have any concept of data types.

- All data is treated as binary data.


## Unprintable characters

- We can think of binary data as a sequence of bytes.

- If we use utf8 or 8-bit ASCII, then we can represent binary data as a Python string.

- Byte values which do not have an associated ASCII character are shown as [hexadecimal](https://en.wikipedia.org/wiki/Hexadecimal) values.

- It is very easy to [convert between Hexadecimal and binary](http://www.ascii.cl/conversion.htm). 


In [4]:
chr(97)

'a'

In [5]:
ord('a')

97

In [6]:
chr(0)

'\x00'


- We can write non-printable characters by prefixing the hexadecimal value with `\x<value>`.

In [7]:
ord('\x00')

0

In [8]:
ord('\x10')

16

## Sequences of bytes

- We can represent any 8-bit value as a character in Python.

- We can represent arbitrary binary data as an ordered sequence of characters.


- This is simply a string.

- For example, the sequence of bytes `0x0010` can be represented:


In [9]:
binary_data = '\x00\x10'
binary_data

'\x00\x10'

In [10]:
binary_data[0]

'\x00'

## Lexicographic ordering of binary data

- We can compare strings according to their alphabetic ordering; e.g.:
    
    

In [11]:
'alpha' < 'beta'

True

In [12]:
'smith' < 'steve'

True

- This still works with unprintable characters:

In [13]:
x = '\x0f\x09\x05\x10\x04'
y = '\x0f\x10\x01\x12\x01'


In [14]:
print type(x)
print type(y)

<type 'str'>
<type 'str'>


In [15]:
x < y

True

### Indexing binary data

- Binary data can be considered as a sequence of bits.

- Therefore it can be considered as a sequence of bytes.

- Therefore binary data can be represented as a string.

- Therefore binary data can be compared and sorted.

- Therefore we can order binary data and store it in, e.g. a tree.

- Therefore binary data can be *indexed*.

### Converting to and from binary data

- All values in Python are ultimately stored as binary data.

- This is usually hidden from the programmer.

- When working with HBase we will often need to deal explicitly with the binary representation.

- This is sometimes called *serialization*.

- We can use Python's [struct class](https://docs.python.org/2/library/struct.html) to do this.

In [1]:
import struct
import binascii

data = 3.141592653589793
float64s = struct.Struct('d')
pi_in_binary = float64s.pack(data)
pi_in_binary

'\x18-DT\xfb!\t@'

- Because some of the characters are printable, it is hard to interpret the above string.

- When dealing with binary data we often want to sequence of bits, or more conveniently the sequence of bytes represented in hexadecimal.

- The `hexlify()` function in the `binascii` module will do this:

In [2]:
binascii.hexlify(pi_in_binary)

'182d4454fb210940'

In [18]:
import struct
import binascii

data = 5
integer64s = struct.Struct('l')
binary_data = integer64s.pack(data)
binascii.hexlify(binary_data)

'0500000000000000'

In [19]:
data = -5
integer64s = struct.Struct('l')
binary_data = integer64s.pack(data)
binascii.hexlify(binary_data)

'fbffffffffffffff'

## Deserializing


In [3]:
original_data = float64s.unpack(pi_in_binary)
original_data

(3.141592653589793,)

In [4]:
original_value = original_data[0]
original_value

3.141592653589793

## Big- and Little-Endian formats

- When we use decimal, by convention we write the most-significant digit on the left, and proceed towards the least-significant digit on the right.

- This is called a [Big-endian](https://en.wikipedia.org/wiki/Endianness#Big-endian) format. 

- We can also use the opposite convention, which is [Little-endian](https://en.wikipedia.org/wiki/Endianness#Little-endian).

- Different CPUs use different formats.

- Intel processors use the Little-endian format.

- The convention used by the CPU is called the *native* format.

- If we want to make our data portable between machines, e.g. transmit it over the network, we may have to specify a particular convention.

## Endian conversions in Python



In [20]:
big_endian_short_int = struct.Struct('>h')
v = big_endian_short_int.pack(1)
binascii.hexlify(v)

'0001'

In [21]:
little_endian_short_int = struct.Struct('<h')
v = little_endian_short_int.pack(1)
binascii.hexlify(v)

'0100'

In [22]:
big_endian_short_int = struct.Struct('>h')
v = big_endian_short_int.pack(-2)
binascii.hexlify(v)

'fffe'

In [23]:
little_endian_short_int = struct.Struct('<h')
v = little_endian_short_int.pack(-2)
binascii.hexlify(v)

'feff'

In [26]:
type(original_value)

float

## Serializing multiple values

In [27]:
import struct
import binascii

values = (1, 'hello', 2.7)
s = struct.Struct('I 5s d')
serialized = s.pack(*values)

print 'Original values:', values
print 'Format string  :', s.format
print 'Uses           :', s.size, 'bytes'
print 'Packed Value   :', binascii.hexlify(serialized)

Original values: (1, 'hello', 2.7)
Format string  : I 5s d
Uses           : 24 bytes
Packed Value   : 0100000068656c6c6f000000000000009a99999999990540


In [28]:
s.unpack(serialized)

(1, 'hello', 2.7)

## HappyBase Prerequisites

1. Install the [`happybase`](http://happybase.readthedocs.org/en/latest/) package:
~~~bash
conda install -c https://conda.anaconda.org/auto \
        happybase
~~~
2. Start HBase:
~~~bash
cd hbase-1.1.2/bin
./start-hbase.sh
~~~
3. Start the HBase Thrift server:
~~~
./hbase thrift start
~~~

- Once the thrift server is up and running you can obtain a connection from Python:

In [29]:
import happybase

host = '127.0.0.1'
connection = happybase.Connection(host)

## Creating a table

In [30]:
table_name = 'my_table'
connection.create_table(table_name,
                        { 'color': dict(max_versions=10),
                          'shape': dict(max_versions=1)
                         })

## Put



In [31]:
integer16s = struct.Struct('h')
four_as_bytes = integer16s.pack(4) 
four_as_bytes

'\x04\x00'

In [32]:
table = connection.table('my_table')
table.put('first', {'shape:square': four_as_bytes} )

## Batch operations

In [33]:
b = table.batch()
b.put('first', {
        'color:red': '\xff\x00\x00', 
        'color:blue': '\x00\x00\xff', 
        'color:yellow': '\xff\xff\x00'
        })
b.put('second', {'shape:triangle': integer16s.pack(3)})
b.send()

## Get

To retrieve a particular cell:

In [34]:
result = table.row('first', columns=['shape:square'])
result

{'shape:square': '\x04\x00'}

In [35]:
binary_data = result['shape:square']

In [36]:
number_of_sides = integer16s.unpack(binary_data)[0]
number_of_sides

4

In [37]:
type(number_of_sides)

int

## Retrieving multiple columns



In [38]:
result = table.row('first', columns=['shape:square', 'color:blue'])
result

{'color:blue': '\x00\x00\xff', 'shape:square': '\x04\x00'}

- We can also retrieve an entire column-family:

In [39]:
result = table.row('first', columns=['color'])
result

{'color:blue': '\x00\x00\xff',
 'color:red': '\xff\x00\x00',
 'color:yellow': '\xff\xff\x00'}

- Or an entire row:

In [40]:
table.row('first')

{'color:blue': '\x00\x00\xff',
 'color:red': '\xff\x00\x00',
 'color:yellow': '\xff\xff\x00',
 'shape:square': '\x04\x00'}

## Scanning a table


In [41]:
for row_key, data in table.scan():
    print row_key, data

first {'color:blue': '\x00\x00\xff', 'color:red': '\xff\x00\x00', 'shape:square': '\x04\x00', 'color:yellow': '\xff\xff\x00'}
second {'shape:triangle': '\x03\x00'}


In [42]:
table.row('first', columns=['shape:square', 'shape:triangle'])

{'shape:square': '\x04\x00'}

## Using URLs as row keys

It is very common to use a URL as the row key.

In [43]:
urls = [
    "http://www.google.com/",
"http://www.baidu.com/",
"http://www.facebook.com/",
"http://www.youtube.com/",
"http://www.yahoo.com/",
"http://www.wikipedia.org/",
"http://www.taobao.com/",
"http://www.qq.com/",
"http://www.amazon.com/",
"http://www.live.com/",
"http://www.twitter.com/",
"http://www.weibo.com/",
"http://www.google.co.in/",
"http://www.tmall.com/",
"http://www.linkedin.com/",
"http://www.blogspot.com/",
"http://www.google.co.jp/",
"http://www.google.de/"
    ]

 ### Reversing URLs
 
 - It is very useful to reverse the order of the domain names when storing them in an ordered-sequence.
 
 - This will allow us to retreive, e.g. all of the ".com" URLs using a scan on a *partial key*.
 
 - Let's parse the URLs and then reverse the domain names, and then store the result in a list with the reversed domain and the original URL.

In [44]:
import urlparse as up

def reverse_domain(dom):
    result = dom.split('.')
    result.reverse()
    return reduce(lambda x, y: x + '.' + y, result)

reversed_domains = \
    [(reverse_domain(up.urlparse(i).netloc), i) for i in urls]
reversed_domains

[('com.google.www', 'http://www.google.com/'),
 ('com.baidu.www', 'http://www.baidu.com/'),
 ('com.facebook.www', 'http://www.facebook.com/'),
 ('com.youtube.www', 'http://www.youtube.com/'),
 ('com.yahoo.www', 'http://www.yahoo.com/'),
 ('org.wikipedia.www', 'http://www.wikipedia.org/'),
 ('com.taobao.www', 'http://www.taobao.com/'),
 ('com.qq.www', 'http://www.qq.com/'),
 ('com.amazon.www', 'http://www.amazon.com/'),
 ('com.live.www', 'http://www.live.com/'),
 ('com.twitter.www', 'http://www.twitter.com/'),
 ('com.weibo.www', 'http://www.weibo.com/'),
 ('in.co.google.www', 'http://www.google.co.in/'),
 ('com.tmall.www', 'http://www.tmall.com/'),
 ('com.linkedin.www', 'http://www.linkedin.com/'),
 ('com.blogspot.www', 'http://www.blogspot.com/'),
 ('jp.co.google.www', 'http://www.google.co.jp/'),
 ('de.google.www', 'http://www.google.de/')]

### Storing the URLs in the database

- We will now create a simple table called `pages` with a single column family `d`:

In [45]:
connection.create_table('pages', { 'd': dict() })

- We will store the details of each URL on each row.

- We will use a single column `times_crawled` to represent the number of times we have crawled the URL.

In [46]:
integer64s = struct.Struct('I')
pages_table = connection.table('pages')
b = pages_table.batch()
for (reversed_domain, url) in reversed_domains:
    b.put(reversed_domain, {'d:url': str(url), 'd:times_crawled': integer64s.pack(0)})
b.send()

In [47]:
def to_int(bytes):
    return integer64s.unpack(bytes)[0]

for (url, data) in pages_table.scan():
    print data['d:url'], "was crawled", \
        to_int(data['d:times_crawled']), "times"

http://www.amazon.com/ was crawled 0 times
http://www.baidu.com/ was crawled 0 times
http://www.blogspot.com/ was crawled 0 times
http://www.facebook.com/ was crawled 0 times
http://www.google.com/ was crawled 0 times
http://www.linkedin.com/ was crawled 0 times
http://www.live.com/ was crawled 0 times
http://www.qq.com/ was crawled 0 times
http://www.taobao.com/ was crawled 0 times
http://www.tmall.com/ was crawled 0 times
http://www.twitter.com/ was crawled 0 times
http://www.weibo.com/ was crawled 0 times
http://www.yahoo.com/ was crawled 0 times
http://www.youtube.com/ was crawled 0 times
http://www.google.de/ was crawled 0 times
http://www.google.co.in/ was crawled 0 times
http://www.google.co.jp/ was crawled 0 times
http://www.wikipedia.org/ was crawled 0 times


## Partial key scans

- We can now retreive URLs for a top-level domain by specifying a *partial key*.

- Here we specify a row *prefix* parameter of `com.`

- Because we have reversed the domain names this will allow us to retrieve particular top-level domains.

In [48]:
for url, data in pages_table.scan(row_prefix="com."):
    print data['d:url'], "was crawled", \
        to_int(data['d:times_crawled']), "times"

http://www.amazon.com/ was crawled 0 times
http://www.baidu.com/ was crawled 0 times
http://www.blogspot.com/ was crawled 0 times
http://www.facebook.com/ was crawled 0 times
http://www.google.com/ was crawled 0 times
http://www.linkedin.com/ was crawled 0 times
http://www.live.com/ was crawled 0 times
http://www.qq.com/ was crawled 0 times
http://www.taobao.com/ was crawled 0 times
http://www.tmall.com/ was crawled 0 times
http://www.twitter.com/ was crawled 0 times
http://www.weibo.com/ was crawled 0 times
http://www.yahoo.com/ was crawled 0 times
http://www.youtube.com/ was crawled 0 times


In [49]:
for url, data in pages_table.scan(row_prefix="de."):
    print data['d:url'], "was crawled", \
        to_int(data['d:times_crawled']), "times"

http://www.google.de/ was crawled 0 times
